In [1]:
# importar librerias
import pandas as pd
import requests
import json

# Pregunta 1

In [2]:
# obtenemos el total de pokemon de 'count'
r = requests.get('https://pokeapi.co/api/v2/pokemon/').json()
count = r["count"]

In [3]:
# obtenemos todos los pokemon
r = requests.get(f'https://pokeapi.co/api/v2/pokemon/?limit={count}').json()

In [4]:
# generamos dataframe con los pokemon y sus nombres
df = pd.DataFrame(r['results'])

In [5]:
# filtramos por el criterio de la pregunta 1
df_filtered = df[(df.name.str.contains('at')) & (df.name.str.count('a') == 2)]
df_filtered.head()

,name,url
19,raticate,https://pokeapi.co/api/v2/pokemon/20/
159,feraligatr,https://pokeapi.co/api/v2/pokemon/160/
379,latias,https://pokeapi.co/api/v2/pokemon/380/
484,heatran,https://pokeapi.co/api/v2/pokemon/485/
503,patrat,https://pokeapi.co/api/v2/pokemon/504/


In [6]:
# obtenemos el resultado de la pregunta 1
df_filtered.count()[0]

14

# Pregunta 2

In [7]:
# Creamos un dataframe con todos los pokemones y sus egg groups
pokemon = 'raichu'
count = requests.get('https://pokeapi.co/api/v2/egg-group/').json()['count']

df_egg_groups = pd.DataFrame()
for i in range(1, count+1):
    r = requests.get(f'https://pokeapi.co/api/v2/egg-group/{i}').json()
    df_aux = pd.DataFrame(r['pokemon_species'])
    df_aux['group'] = r['name']
    df_egg_groups = df_egg_groups.append(df_aux, ignore_index = True)

df_egg_groups.head()

,name,url,group
0,bulbasaur,https://pokeapi.co/api/v2/pokemon-species/1/,monster
1,ivysaur,https://pokeapi.co/api/v2/pokemon-species/2/,monster
2,venusaur,https://pokeapi.co/api/v2/pokemon-species/3/,monster
3,charmander,https://pokeapi.co/api/v2/pokemon-species/4/,monster
4,charmeleon,https://pokeapi.co/api/v2/pokemon-species/5/,monster


In [8]:
# Filtramos por pokemones segun si comparten un grupo con el pokemon buscado (raichu)
aux = df_egg_groups.loc[df_egg_groups['name'].str.lower() == pokemon, ['group']]
df_egg = df_egg_groups[df_egg_groups['group'].isin(aux['group'].values)]

# eliminamos duplicados si hay (interesa solo los nombres, nos deshacemos de las otras columnas por practicalidad)
df_egg = df_egg.drop(columns=['group', 'url'])
df_egg = df_egg.drop_duplicates()
df_egg.head()

,name
308,rattata
309,raticate
310,ekans
311,arbok
312,pikachu


In [9]:
# Contamos las filas, es decir, todas las especies que pueden procrear con raichu
# Resultado
df_egg.count()[0]

258

# Pregunta 3

In [10]:
# Buscamos a todos los pokemon tipo fighting (con id menor o igual a 151)
fighters = requests.get('https://pokeapi.co/api/v2/type/fighting').json()
fighters = fighters['pokemon']
fighters = pd.DataFrame(fighters)
fighters = pd.concat([fighters.drop(['pokemon'], axis=1), fighters['pokemon'].apply(pd.Series)], axis=1)

# de 'url' sacamos el id
fighters['num'] = fighters.apply(lambda x: int(x['url'].split('/')[6]), axis=1)
fighters = fighters[fighters['num'] <= 151]
fighters

,slot,name,url,num
0,1,mankey,https://pokeapi.co/api/v2/pokemon/56/,56
1,1,primeape,https://pokeapi.co/api/v2/pokemon/57/,57
2,2,poliwrath,https://pokeapi.co/api/v2/pokemon/62/,62
3,1,machop,https://pokeapi.co/api/v2/pokemon/66/,66
4,1,machoke,https://pokeapi.co/api/v2/pokemon/67/,67
5,1,machamp,https://pokeapi.co/api/v2/pokemon/68/,68
6,1,hitmonlee,https://pokeapi.co/api/v2/pokemon/106/,106
7,1,hitmonchan,https://pokeapi.co/api/v2/pokemon/107/,107


In [11]:
# Iteramos sobre los pokemones seleccionados y actualizamos valores de peso max y min
pmin = 99999999
pmax = 0
for row in fighters.itertuples():
    r = requests.get(row[3]).json()
    if r['weight'] < pmin:
        pmin = r['weight']
    if r['weight'] > pmax:
        pmax = r['weight']

# Finalmente creamos la lista final
respuesta = [pmax, pmin]
respuesta


[1300, 195]